In [1]:
# Imports

import pandas as pd
import re
import numpy as np
import cv2
import os
os.environ['QT_QPA_PLATFORM'] = 'xcb'
from scipy.ndimage import median_filter
from PIL import Image
from sklearn.cluster import DBSCAN
from skimage.morphology import erosion, square

In [19]:
# Use bag file after converting it to csv using rosbag
# Replace 233 with the number of rows in the csv file

main_df = pd.read_csv('Bottle2/events.csv')
print(main_df.shape)

for i in range(0, main_df.shape[0]):
    x = main_df.iloc[i,7]
    df_1 = pd.DataFrame([x], columns=['events'])
    df_1.to_csv(f'Bottle2/Preprocess/df_{i}.csv', index=False)

(172, 8)


In [20]:
for i in range(0, main_df.shape[0]):
    df = pd.read_csv(f'Bottle2/Preprocess/df_{i}.csv')
    def clean_event(event):
       event = re.sub('ts: \n  secs: \d+\n  nsecs:\s+\d+\n', '', event)
       return event
    df['events'] = df['events'].apply(clean_event)
    df.to_csv(f'Bottle2/Preprocess/df_{i}_split.csv', index=False)

In [21]:
for i in range(0, main_df.shape[0]):
    df = pd.read_csv(f'Bottle2/Preprocess/df_{i}_split.csv')
    df = df['events'].str.split(',', expand=True).stack().reset_index(level=1, drop=True).to_frame('events')
    df.to_csv(f'Bottle2/Preprocess/df_{i}_split_rows.csv', index=False)

In [22]:
for i in range(0, main_df.shape[0]):
    df = pd.read_csv(f'Bottle2/Preprocess/df_{i}_split_rows.csv')
    pattern = r"x: (?P<x>\d+)\ny: (?P<y>\d+)\npolarity: (?P<polarity>True|False)"
    df[['x', 'y', 'polarity']] = df['events'].str.extract(pattern)
    first_column = df.columns[0]
    df = df.drop(first_column, axis=1)
    df.to_csv(f'Bottle2/Preprocess/df_{i}_final.csv', index=False)

In [24]:
for i in range(0, main_df.shape[0]):
    df = pd.read_csv(f'Bottle2/Preprocess/df_{i}_final.csv')
    img = np.zeros((640, 480))
    for j in range(0, df.shape[0]):
        if df.iloc[j, 2] == True:
            x = int(df.iloc[j, 0])
            y = int(df.iloc[j, 1])
            img[x, y] = 1
    img = (img * 255).astype(np.uint8)  
    cv2.imwrite(f'Bottle2/Images/img_{i}.png', img)

In [6]:
# Median filter to remove noise
for i in range(0, 233):
    img = Image.open(f'raw_image/img_{i}.png')
    image_array = np.array(img)
    filtered_image_array = erosion(image_array, square(4))

    white_pixel_indices = np.where(filtered_image_array == 255)
    
    if white_pixel_indices[0].size > 0:
        min_y, max_y = np.min(white_pixel_indices[0]), np.max(white_pixel_indices[0])
        min_x, max_x = np.min(white_pixel_indices[1]), np.max(white_pixel_indices[1])

        cv2.rectangle(filtered_image_array, (min_x, min_y), (max_x, max_y), (255, 0, 0), 2)

        filtered_image = Image.fromarray(filtered_image_array)
        filtered_image.save(f'annotated_image/img_{i}.png')
    else:
        img.save(f'annotated_image/img_{i}.png')
  

In [8]:
# Erosion and DBSCAN
num_clusters = []
cluster_data = []

for i in range(0, 171):
    img = Image.open(f'Bottle2/Images/img_{i}.png')
    image_array = np.array(img)
    eroded_image_array = erosion(image_array, square(3))
    white_pixel_indices = np.where(eroded_image_array == 255)
    
    if white_pixel_indices[0].size > 0:
        data = np.array(list(zip(white_pixel_indices[1], white_pixel_indices[0])))

        dbscan = DBSCAN(eps=90, min_samples=3)
        dbscan.fit(data)

        unique_labels = set(dbscan.labels_)
        num_clusters.append(len(unique_labels) - (1 if -1 in unique_labels else 0))

        cluster_data.append((data, dbscan.labels_, unique_labels))
    else:
        img.save(f'Bottle2/real_cluster/img_{i}.png')
        cluster_data.append((None, None, None))

In [10]:
for i, cluster in enumerate(cluster_data):
    if cluster is None:  
        continue

    data, labels, unique_labels = cluster  
    img = Image.open(f'Bottle2/Images/img_{i}.png')
    image_array = cv2.cvtColor(np.array(img), cv2.COLOR_GRAY2BGR)

    if unique_labels is not None:  
        for k in unique_labels:
            if k == -1:
                col = [0, 0, 0, 1]
            else:
                col = (0, 0, 255)  

            class_member_mask = (labels == k)

            xy = data[class_member_mask]
            if k != -1:  
                x, y, w, h = cv2.boundingRect(xy.astype(np.float32))
                cv2.rectangle(image_array, (x, y), (x + w, y + h), col, 2)
            else:
                for point in xy:
                    cv2.circle(image_array, (point[0], point[1]), 2, col, -1)

    filtered_image = Image.fromarray(image_array)
    filtered_image.save(f'Bottle2/real_cluster/img_{i}.png')

for i, num in enumerate(num_clusters):
    print(f'Image {i}: {num} clusters')

Image 0: 0 clusters
Image 1: 1 clusters
Image 2: 1 clusters
Image 3: 1 clusters
Image 4: 1 clusters
Image 5: 1 clusters
Image 6: 1 clusters
Image 7: 1 clusters
Image 8: 1 clusters
Image 9: 1 clusters
Image 10: 1 clusters
Image 11: 1 clusters
Image 12: 1 clusters
Image 13: 1 clusters
Image 14: 2 clusters
Image 15: 3 clusters
Image 16: 3 clusters
Image 17: 3 clusters
Image 18: 2 clusters
Image 19: 2 clusters
Image 20: 1 clusters
Image 21: 2 clusters
Image 22: 1 clusters
Image 23: 2 clusters
Image 24: 2 clusters
Image 25: 1 clusters
Image 26: 2 clusters
Image 27: 1 clusters
Image 28: 2 clusters
Image 29: 3 clusters
Image 30: 2 clusters
Image 31: 1 clusters
Image 32: 2 clusters
Image 33: 2 clusters
Image 34: 2 clusters
Image 35: 2 clusters
Image 36: 1 clusters
Image 37: 1 clusters
Image 38: 1 clusters
Image 39: 1 clusters
Image 40: 1 clusters
Image 41: 1 clusters
Image 42: 1 clusters
Image 43: 1 clusters
Image 44: 1 clusters
Image 45: 1 clusters
Image 46: 2 clusters
Image 47: 2 clusters
Im